# Technical Portfolio Analyzer

This Jupyter Notebook will outline the exploration, developement and clean-up of the appliation code

### Table of Contents
- Data connectors
- Data query and clean-up
- Analytical methods
    - Technical
    - Fundamental
    - Traditional
    
---

- Visualization
- Dashboard (GUI)
    - GUI
    - Data filtration methods

### Dependencies

In [1]:
# Data analytics
import pandas as pd
import numpy as np
import panel as pn
# import bs4 as bs

# Visualization
# pn.extension('plotly')
# import plotly.express as px
# import hvplot.pandas
import matplotlib.pyplot as plt

# System
import os
import time
from pathlib import Path
from dotenv import load_dotenv
import requests

# Finance
import alpaca_trade_api as tradeapi
import quandl as ql
import finnhub

import warnings
warnings.filterwarnings('ignore')

## Data Connections
- Static Data Connections
- Dynamic Data Connections

### Static Data Connections

#### Stock Ticker Lists

In [2]:
# Get tickers within S&P500 index - test dataset
sp500_tickers_path = Path('resources/sp500_tickers.csv')
sp500_tickers = pd.read_csv(sp500_tickers_path)

sp500_tickers_test_path = Path('resources/sp500_tickers_test.csv')
sp500_tickers_test = pd.read_csv(sp500_tickers_test_path)

In [3]:
# Get fundamental data from csv
stock_fundamentals_data_path = Path("resources/fundamental_data.csv")
stock_fundamentals_df = pd.read_csv(stock_fundamentals_data_path)

### Dynamic Data Connections

#### Stock Price Data

In [4]:
# Alpaca API connector
load_dotenv("../resources/api_keys.env")

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version = "v2"
)

#### Fundamental Stock Data

In [5]:
# FinnHub API connector
load_dotenv("../resources/api_keys.env")

# Set FinnBug API key
finnhub_api_key = os.getenv("FINNHUB_API_KEY")

# Create FinnHub API object
finnhub_client = finnhub.Client(api_key=finnhub_api_key)

# Method to obtain json data from Polygon API
def finnhub_data(ticker):
    
    data = finnhub_client.company_basic_financials(ticker, "")
    data_df = pd.DataFrame(data)
    time.sleep(1)
    
    return data_df

#### Bond Data

In [6]:
# Treasury bonds
def treasury_data():
    return ql.get("USTREASURY/YIELD")

#### Crypto Data

In [7]:
# Crypto connector URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

# Build out the crypto connector here

## Data Parsing

### Stock Data

In [8]:
# Get prices for tickers withing a given index or sector
def stock_prices(tickers_df, start_date, end_date):
    '''Returns pd.DataFrame with prices for the given tickers
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    start_date : str() - string with date in following format YYYY-MM-DD
    end_date: str() - string with date in following format YYYY-MM-DD 
    
    
    Returns
    -------
    result_df : pd.DataFrame with securities price data
    '''
    
    # Get list of tickers from the tickers_df DataFrame
    tickers = list(tickers_df["Symbol"])
    
    # Parse start and end dates
    start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()
    
    # Connect to Alpaca API and get data
    """Condition handling: 
        a. Alpaca API 422 Client Error if more than 100 tickers are passed - COMPLETE
        b. Alpaca API data max row limit of 1000 - PENDING"""
    
    
    # a. Alpaca API condition handling, sending 100 tickers at a time
    # Declate a pd.DataFrame
    result_df = pd.DataFrame()
    tickers_n = 50
    
    for i in range(0, len(tickers), tickers_n):
        # Slice the ticker list into lists of 50 tickers
        sliced_tickers = tickers[i:i + tickers_n] 
        
        temp_df = api.get_barset(
        sliced_tickers,
        timeframe = "1D",
        start = start_date,
        end = end_date,
        limit = 1000).df

        # Append temporary dataframe to result_df
        result_df = pd.concat([result_df, temp_df], axis = "columns", join = "outer")
        time.sleep(0.1)
        
    return result_df

# start_date = '2020-01-01'
# end_date = '2020-02-28'
# tickers_df = sp500_tickers
# stocks = stock_prices(tickers_df, start_date, end_date)
# stocks.to_csv("resources/price_data.csv")
# stocks.head()



# Get stock fundamental data from FinnHub
def generate_stock_fundamentals(tickers_df):
    '''Returns pd.DataFrame with fundamentals of tickers within tickers_df
    
    ...
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
        
    Returns
    -------
    result_df : pd.DataFrame - securities fundamental data
    '''
    
    result_df = pd.DataFrame()
    
    for ticker in tickers_df['Symbol']:
        fundamental_data = finnhub_data(ticker)
        result_df = pd.concat([result_df, fundamental_data], axis='rows', join="outer")
        
    # Parse the dataframe
    result_df = result_df.reset_index().set_index('symbol')
    result_df = result_df.drop('metricType', 1)
    result_df.columns = ['metric_type', 'metric', 'series']
    
    return result_df

# fundamental_data = stock_fundamentals(sp500_tickers_test)
# fundamental_data.head()
        

    
# Refresh stock fundamental data csv
def refresh_fundamentals_csv(tickers_df):
    '''Returns pd.DataFrame with fundamentals of tickers within tickers_df
    and/or calls to generate a refreashed dataset
    
    ...
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
    '''
    
    result_df = stock_fundamentals(tickers_df)
    result_df.to_csv("resources/fundamental_data.csv")
    
# refresh_fundamentals_csv(sp500_tickers)

## Computational Methods

### Technical

In [9]:
# please start with RSI and MACD
# your method must take entire dataframe and add a RSI or MACD column to it
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)


def rsi(df, days):
    '''Returns a pd.DataFrame with RSI column appended
    
    ...
    
    Parameters
    ----------
    df : pd.DataFrame - dataframe to be processed
    days : int() - numbers of days for RSI calcualtion
    
    Returns
    -------
    result_df : pd.DataFrame - dataframe with RSI column appended, calcualted daily for 
        timeperiod specified by days
    '''
    
    result_df = pd.DataFrame()
    
    # Your code
    
    return result_df

def macd(df, short_ema, long_ema):
    '''Description...
    
    ...
    
    Parameters
    ----------
    df : pd.DataFrame - dataframe to be processed
    short_ema : int() - short-term EMA for MACD calculation
    long_ema : int() - long-term EMA for MACD calculation
    
    Returns
    -------
    result_df : pd.DataFrame - dataframe with MACD column appended, calcualted daily for 
        timeperiod specified by days
    '''
    pass

# Method test

### Fundamental

In [66]:
# Please start with PE, EPS, and Dividned Info.
# your method must take entire dataframe and add a PE, EPS, and Dividned Info columns to it
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)

# Fundamental data filter
def fundamental_data_query(tickers_df, fundamental_indicator_key, lower_bound = -1000000, upper_bound = 1000000):
    '''Returns a pd.DataFrame of fundamental data filtered by user input range
    
    ...
    
    Parameters
    ----------
    tickers_df : pd.DataFrame - dataframe to be processed, contains ticker and fundamental data
    fundamental_indicator_key : str() - keyword for fundamental indicator requested
    
        Fundamental indicator keys ->
        
        P/E Ratio : [pe_ratio]
        EPS (Earnings per Share) : [eps]
        Annual Dividend : [dividend]
        Beta (vs. S&P 500) : [beta]
        EBIDT : [ebidt]
        Quick Ratio : [quick_ratio]
        3 Year Revenue Growth : [rev_growth]
        Free Cash Flow : [cash_flow]
    
    lower_bound : int() or float() - lower bound for fundamental value filter, default = -1000000
    upper_bound : int() or float() - upper bound for fundamental value filter, default = 1000000
     
     
     
    Returns
    -------
    result_df : pd.DataFrame - dataframe with ticker and filtered fundamental data
    '''

    fund_indicators_dict = {
        'pe_ratio' : 'peNormalizedAnnual',
        'eps' : 'epsNormalizedAnnual',
        'dividend' : 'dividendsPerShareTTM',
        'beta' : 'beta',
        'ebidt' : 'ebitdPerShareTTM',
        'quick_ratio' : 'quickRatioAnnual',
        'rev_growth' : 'revenueGrowth3Y',
        'free_cash_flow' : 'freeCashFlowAnnual'   
    }
    
    # Declate tickers list
    tickers_list = tickers_df['Symbol']
    
    # Declare fundamental data df and filter by ticker df
    data_df = stock_fundamentals_df[stock_fundamentals_df.symbol.isin(tickers_list)]
    data_df = data_df.set_index(['symbol'])
    
    # Extract requested fundamental data
    result_df = data_df[data_df['metric_type'] == fund_indicators_dict[fundamental_indicator_key]]
    
    # Convert all df values to numeric
    result_df['metric'] = result_df['metric'].apply(pd.to_numeric)
    
    # Filter fundamental data by user input range (lower_bound and upper_bound)
    result_df = result_df[(result_df['metric'] > lower_bound) & (result_df['metric'] < upper_bound)]
    
    # Clean up result_df
    result_df = result_df.drop(columns = ['metric_type', 'series'])
    result_df = result_df.rename(columns = {'symbol' : 'ticker', 
                                'metric' : fundamental_indicator_key})
    
    
    return result_df


# Method test
# print(fundamental_data_query(sp500_tickers, 'pe_ratio', 10, 11))
# help(fundamental_data_query)

        pe_ratio
symbol          
BIIB    10.50522
INTC    10.60050
KR      10.66000
MTB     10.88400
PGR     10.67319


### Traditional

In [ ]:
# Please start with calculating the Sharpe Ratio Calculation
# your method must take entire dataframe and return a dataframe with ticker and sharpe ratio for the given time
# please keep in mind that the dataframe will contain multiple tickers
# follow specs for method writing outlined below, feel free to expand and improve
# document your method with docstring
# document theory for your method in readme (see the README.md for example)

# Method test

## Visualization

In [ ]:
# your code here

## Dashboard

In [ ]:
# your code here